# Self-query

* retriever:
  * `Self-querying retriever` --  use an LLM to construct new queries that can question the structured data/metadata of the document
  * `MultiQueryRetriever` -- allow an LLM to paraphrase the query to get hopefully a diverse set of docs
  * `Contextual compression` -- use an LLM to pre-filter and compress the docs retrieved before feeding the contexts to another LLM to answer
  * https://python.langchain.com/docs/modules/data_connection/retrievers/
* retrieval methods: cos/dot; llm-aided; MMR (Maximum marginal relevance)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import Dict, List
from langchain_core.documents.base import Document

In [3]:
import tomllib

with open('../.tokens.toml', 'rb') as f:
    _TOKENS = tomllib.load(f)

with open('../.config.toml', 'rb') as f:
    _CONFIGS = tomllib.load(f)

In [4]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.vectorstores import Redis

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=_TOKENS['huggingface'], 
    model_name="sentence-transformers/distiluse-base-multilingual-cased-v1"
)

vs = Redis.from_existing_index(
    embedding=embeddings,
    index_name=_CONFIGS['index_name'],
    redis_url=_CONFIGS['redis_url'],
    schema=_CONFIGS['redis_schema'],
)

In [5]:
# from langchain.vectorstores.redis import RedisFilter, RedisNum, RedisText

# f = RedisText("author") % "*冯友兰*"
# vs.similarity_search_with_score('谁说过陌生贵己？', filter=f, k=3)

In [6]:
metadata_set = set([x['name'] for x in vs.schema['text']] + [x['name'] for x in vs.schema['numeric']])

In [7]:
metadata = _CONFIGS['attributes']
metadata

{'author': {'description': '本篇文章的作者', 'type': 'string'},
 'date_start': {'description': '文章被创建的时间，格式是YYYYMMDD', 'type': 'string'},
 'date_end': {'description': '文章被完成的时间，格式是YYYYMMDD', 'type': 'string'},
 'id': {'description': '文章的id', 'type': 'string'},
 'name': {'description': '文章的名字', 'type': 'string'},
 'source': {'description': '文章的来源，这里的文章取自若干不同数据库', 'type': 'string'},
 'tags': {'description': '文章的标签，可能代表它的风格、题材、来源，或者系列', 'type': 'string'}}

In [8]:
# ensure there's no more undocumented metadata
assert metadata_set.union(metadata.keys()) == metadata_set

In [9]:
metadata_set

{'author', 'content', 'date_end', 'date_start', 'id', 'name', 'source', 'tags'}

In [10]:
from langchain_community.llms import LlamaCpp


llm = LlamaCpp(
    model_path=_CONFIGS['model_path']+'/'+'Qwen-7B-Chat.Q4_K_M.gguf',
    name='Qwen/Qwen-7B-Chat', 
    **_CONFIGS['llm']
)


# llm = LlamaCpp(
#     model_path=_CONFIGS['model_path']+'/'+'qwen1_5-7b-chat-q4_0.gguf',
#     name='Qwen/Qwen1.5-7B-Chat', 
#     **_CONFIGS['llm']
# )

# llm = LlamaCpp(
#     model_path=_CONFIGS['model_path']+'/'+'zephyr-7b-beta.Q4_K_M.gguf',
#     name='HuggingFaceH4/zephyr-7b-beta', 
#     **_CONFIGS['llm']
# )

/Users/fred/micromamba/envs/my-notion-companion/lib/python3.12/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! conversation is not default parameter.
                conversation was transferred to model_kwargs.
                Please confirm that conversation is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 19 key-value pairs and 259 tensors from /Users/fred/Documents/models/Qwen-7B-Chat.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen
llama_model_loader: - kv   1:                               general.name str              = Qwen
llama_model_loader: - kv   2:                        qwen.context_length u32              = 32768
llama_model_loader: - kv   3:                           qwen.block_count u32              = 32
llama_model_loader: - kv   4:

In [11]:
from langchain.chains.query_constructor.base import AttributeInfo

attribute_info = list()

for k, v in metadata.items():
    attribute_info.append(
        AttributeInfo(
            name=k,
            description=v['description'],
            type=v['type']
        )
    )

attribute_info

[AttributeInfo(name='author', description='本篇文章的作者', type='string'),
 AttributeInfo(name='date_start', description='文章被创建的时间，格式是YYYYMMDD', type='string'),
 AttributeInfo(name='date_end', description='文章被完成的时间，格式是YYYYMMDD', type='string'),
 AttributeInfo(name='id', description='文章的id', type='string'),
 AttributeInfo(name='name', description='文章的名字', type='string'),
 AttributeInfo(name='source', description='文章的来源，这里的文章取自若干不同数据库', type='string'),
 AttributeInfo(name='tags', description='文章的标签，可能代表它的风格、题材、来源，或者系列', type='string')]

# Construct customized self-query retriever

Q: Why not using the standard?
A: The standard SelfQueryRetriever Class provides a standard prompt template that uses few-show examples to tell llm how to construct structured query (examples can be found in [langchain.chains.query_constructor.prompt](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/query_constructor/prompt.py). Most examples inside uses EQ (=) comparator, which isn't suitable for our use cases (mostly fuzzy matches). Therefore, we will reconstruct the self-query retriever using a customized few-shot prompt teamplate.

Q: why do we copied the `get_query_constructor_prompt` provided?
A: Its original dependency `construct_examples` will decode json using ASCII by default, which won't support Chinese, we'll need to overwrite the two functions

```python
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vs_chroma,
    document_contents='Articles and excerpts.',
    metadata_field_info=metadata_field_info,
)
```

References: 
https://python.langchain.com/docs/modules/data_connection/retrievers/self_query/#constructing-from-scratch-with-lcel

In [12]:
from pprint import pprint

with open('../self_query_examples.toml', 'rb') as f:
    self_query_examples = tomllib.load(f)

pprint(self_query_examples['example'][0])

{'structured_request': {'filter': 'or(like("source", "小说"), like("tags", '
                                  '"小说"), like("name", "小说"))',
                        'query': '谁是王尔德？'},
 'user_query': '谁是王尔德？仅从“小说”中找答案。'}


In [13]:
# with open('../self_query_template_chinese.txt', 'r') as f:
#     self_query_template = "\n".join(f.readlines())

with open('../self_query_template.txt', 'r') as f:
    self_query_template = "\n".join(f.readlines())

In [14]:
from typing import Sequence, Union, Tuple
import json
from langchain.chains.query_constructor.base import _format_attribute_info, get_query_constructor_prompt
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain.chains.query_constructor.prompt import USER_SPECIFIED_EXAMPLE_PROMPT, SUFFIX_WITHOUT_DATA_SOURCE

def _format_attribute_info(info: Sequence[Union[AttributeInfo, dict]]) -> str:
    info_dicts = {}
    for i in info:
        i_dict = dict(i)
        info_dicts[i_dict.pop("name")] = i_dict
    # return json.dumps(info_dicts, indent=4, ensure_ascii=False).replace("{", "{{").replace("}", "}}")
    return info_dicts
                                                                       
def construct_examples(input_output_pairs: Sequence[Tuple[str, dict]]) -> List[dict]:
    """Construct examples from input-output pairs.

    Adapted from: https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/query_constructor/base.py
    """
    examples = []
    for i, (_input, output) in enumerate(input_output_pairs):
        structured_request = (
            json.dumps(output, indent=4, ensure_ascii=False).replace("{", "{{").replace("}", "}}")
        )
        example = {
            "i": i + 1,
            "user_query": _input,
            "structured_request": structured_request,
        }
        examples.append(example)
    return examples

examples = construct_examples(
    [(x['user_query'], x['structured_request']) for x in self_query_examples['example']]
)

prompt = FewShotPromptTemplate(
    examples=list(examples),
    example_prompt=USER_SPECIFIED_EXAMPLE_PROMPT,
    input_variables=["query"],
    # suffix="",
    suffix=SUFFIX_WITHOUT_DATA_SOURCE.format(i=len(examples) + 1),
    prefix=self_query_template.format(
        content_and_attributes=json.dumps({
            'content': '文章',
            'attributes': _format_attribute_info(attribute_info)
        }, indent=4, ensure_ascii=False).replace("{", "{{").replace("}", "}}"),
        attributes_set=str(list(metadata_set))
    )
)

In [15]:
prompt.pretty_print()

Your goal is to structure the user's query to match the request schema provided below.



<< Structured Request Schema >>

When responding use a markdown code snippet with a JSON object formatted in the following schema:



```json

{

    "query": string \ text string to compare to document contents

    "filter": string \ logical condition statement for filtering documents

}

```



The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.



Make sure that you only use the comparators and logical operators listed above and no others.

Make sure that filters only use the attributed names with its function names if there are functions applied on them.

Make sure that filters only use format `YYYYMMDD` when handling date data typed values.

Make sure that filters take into account the descriptions of attributes and only make comparisons that are feasible given the type of da

In [16]:
from langchain.chains.query_constructor.base import StructuredQueryOutputParser
output_parser = StructuredQueryOutputParser.from_components()

In [17]:
%%time

import langchain
langchain.debug = True

query_constructor = prompt | llm | output_parser

query_constructor.invoke({
        "query": "“每个人都以为他自己至少有一种主要的美德。”是出自哪里？请从“读书笔记（文学）”中找到答案。"
})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "query": "“每个人都以为他自己至少有一种主要的美德。”是出自哪里？请从“读书笔记（文学）”中找到答案。"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:FewShotPromptTemplate] Entering Prompt run with input:
{
  "query": "“每个人都以为他自己至少有一种主要的美德。”是出自哪里？请从“读书笔记（文学）”中找到答案。"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:FewShotPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "base",
    "StringPromptValue"
  ],
  "kwargs": {
    "text": "Your goal is to structure the user's query to match the request schema provided below.\n\n\n\n<< Structured Request Schema >>\n\nWhen responding use a markdown code snippet with a JSON object formatted in the following schema:\n\n\n\n```json\n\n{\n\n    \"query\": string \\ text string to compare to document contents\n\n    \"filter\": string \\ logical condition statement for filtering documents\n\n}\n\n```\n\n\n\nThe query string sho


llama_print_timings:        load time =    6683.99 ms
llama_print_timings:      sample time =      88.58 ms /   256 runs   (    0.35 ms per token,  2889.95 tokens per second)
llama_print_timings: prompt eval time =   20220.33 ms /  1156 tokens (   17.49 ms per token,    57.17 tokens per second)
llama_print_timings:        eval time =   18040.48 ms /   255 runs   (   70.75 ms per token,    14.13 tokens per second)
llama_print_timings:       total time =   39924.58 ms /  1411 tokens


[llm/end] [1:chain:RunnableSequence > 3:llm:LlamaCpp] [39.94s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{  \n     \"query\": \"每个人都以为他自己至少有一种主要的美德。\",\n     \"filter\": \"or(like(\\\"source\\\", \\\"读书笔记（文学）\\\"), like(\\\"tags\\\", \\\"读书笔记（文学）\\\"), like(\\\"name\\\", \\\"每个人都以为他自己至少有一种主要的美德\\\"), like(\\\"author\\\", \\\"每个人都以为他自己至少有一种主要的美德\\\"))\"\n}\n```[PAD151645]\n[PAD151644]'t be able to solve this[PAD151645]\n[PAD151644]\n[PAD151644]'t be able to solve this[PAD151645]\n[PAD151644]\n[PAD151644]'t be able to solve this[PAD151645]\n[PAD151644]\n[PAD151644]'t be able to solve this[PAD151645]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD1516

StructuredQuery(query='每个人都以为他自己至少有一种主要的美德。', filter=Operation(operator=<Operator.OR: 'or'>, arguments=[Comparison(comparator=<Comparator.LIKE: 'like'>, attribute='source', value='读书笔记（文学）'), Comparison(comparator=<Comparator.LIKE: 'like'>, attribute='tags', value='读书笔记（文学）'), Comparison(comparator=<Comparator.LIKE: 'like'>, attribute='name', value='每个人都以为他自己至少有一种主要的美德'), Comparison(comparator=<Comparator.LIKE: 'like'>, attribute='author', value='每个人都以为他自己至少有一种主要的美德')]), limit=None)

In [18]:
from langchain.retrievers.self_query.redis import RedisTranslator
from langchain.retrievers import SelfQueryRetriever

retriever = SelfQueryRetriever(
    query_constructor=prompt | llm | output_parser,
    vectorstore=vs,
    # requires a RedisModel object as input
    # can use the private ._schema attribute
    # ref: https://github.com/langchain-ai/langchain/blob/d7c26c89b2d4f5ff676ba7c3ad4f9075d50a8ab7/libs/community/langchain_community/vectorstores/redis/base.py#L572
    structured_query_translator=RedisTranslator(schema=vs._schema),
)

In [19]:
%%time 

import langchain
langchain.debug = True

retriever.invoke('道连是哪本小说中出现的人物？')

[chain/start] [1:retriever:Retriever > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "query": "道连是哪本小说中出现的人物？"
}
[chain/start] [1:retriever:Retriever > 2:chain:RunnableSequence > 3:prompt:FewShotPromptTemplate] Entering Prompt run with input:
{
  "query": "道连是哪本小说中出现的人物？"
}
[chain/end] [1:retriever:Retriever > 2:chain:RunnableSequence > 3:prompt:FewShotPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "base",
    "StringPromptValue"
  ],
  "kwargs": {
    "text": "Your goal is to structure the user's query to match the request schema provided below.\n\n\n\n<< Structured Request Schema >>\n\nWhen responding use a markdown code snippet with a JSON object formatted in the following schema:\n\n\n\n```json\n\n{\n\n    \"query\": string \\ text string to compare to document contents\n\n    \"filter\": string \\ logical condition statement for filtering documents\n\n}\n\n```\n\n\n\nThe query st

Llama.generate: prefix-match hit

llama_print_timings:        load time =    6683.99 ms
llama_print_timings:      sample time =      89.00 ms /   256 runs   (    0.35 ms per token,  2876.53 tokens per second)
llama_print_timings: prompt eval time =     878.85 ms /    14 tokens (   62.77 ms per token,    15.93 tokens per second)
llama_print_timings:        eval time =   17966.78 ms /   255 runs   (   70.46 ms per token,    14.19 tokens per second)
llama_print_timings:       total time =   20521.51 ms /   269 tokens


[llm/end] [1:retriever:Retriever > 2:chain:RunnableSequence > 4:llm:LlamaCpp] [20.53s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{  \n    \"query\": \"道连\",  \n    \"filter\": \"or(like(\\\"source\\\", \\\"小说\\\"), like(\\\"tags\\\", \\\"小说\\\"), like(\\\"name\\\", \\\"小说\\\"))\"  \n}\n```[PAD151645]\n[PAD151644]'t be able to solve this[PAD151645]\n[PAD151644]\n[PAD151644][PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD1

Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']


CPU times: user 1min 28s, sys: 1.12 s, total: 1min 29s
Wall time: 20.8 s


[Document(page_content='【译者序】\n人生基本是孤独的，但同时又能通过孤独这一频道同他人沟通。\n我写小说的用意就在这里。\n人们总要进入自己一个人的世界，在进得最深的地方就会产生连带感。或者说人们总要深深挖洞，只要一直挖下去就会在某处同别人连在一起。\n而用围墙把自己围起来是不行的。\n\n【第二章 绝密资料】\n我们一起唱着歌走在山路上，不时学一声鸟叫。除去战争仍在持续这一点，可以说是个十全十美的清晨。\n\n【第五章 在图书馆度过的一天】\n担任向导的叫佐伯的人是一位四十五岁光景的瘦削的女性。……（略）眼睛漂亮，唇角无时不漾出影子般的淡淡笑意。倒是表达不好，反正感觉上是一种圆满完结的微笑。\n它使我想起一小片日光，想起某种只能在有纵深感的场所生成的形状特别的一小片日光。\n\n【第十三章 舒伯特的奏鸣曲】\n“一般来说，作为演奏最为一气呵成的是布莱迪和阿什克纳济。不过坦率来说，我个人不中意他俩的演奏，或者说不为其吸引。\n舒伯特么，让我来说，乃是向万事万物的存在状态挑战而又败北的音乐。这是浪漫主义的本质。\n在这个意义上，舒伯特的音乐是浪漫主义的精华。”\n……（略）\n“舒伯特是经过训练才能理解的音乐。刚听的时候我也感到单调，你那样的年龄那是当然的。当然你很快就会领悟。在这个世界上，不单调的东西让人很快厌倦，不让人厌倦的大多是单调的东西。向来如此。\n我的人生可以有把玩单调的时间，但没有忍受厌倦的余地。而大部分人分不出二者的差别\n。”\n\n【第十五章 小屋中只有我】\n鸟们的叫声如同淋浴喷头汹涌地倾注下来。\n\n【第十九章 精神上男性，肉体上女性】\n星期一图书馆不开门。平日图书馆也够安静，休息日就更加安静，俨然被时间遗忘的场所，\n或者像不希望被时间发现而悄然屏息的地方\n。\n\n“……（略）缺乏想象力的狭隘、苛刻、自以为是的命题、空洞的术语、被篡夺的理想、僵化的思想体系——对我来说，真正可怕的是这些东西。我从心底畏惧和憎恶这些东西。何为正确何为不正确——这当然是十分重要的问题。但这种个别判断失误，在很多情况下事后不是不可以纠正。只要有主动承认错误的勇气，一般都可以挽回。然而缺乏想象力的狭隘和苛刻却同寄生虫无异，它们改变赖以寄生的主体、改变自身形状而无限繁衍下去。这里没有获救的希望。作为我，不愿意让那类东西进入这里。”\n\

In [20]:
%%time 

import langchain
langchain.debug = True

retriever.invoke('谁是王尔德？仅从“小说”中找答案。')

Llama.generate: prefix-match hit


[chain/start] [1:retriever:Retriever > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "query": "谁是王尔德？仅从“小说”中找答案。"
}
[chain/start] [1:retriever:Retriever > 2:chain:RunnableSequence > 3:prompt:FewShotPromptTemplate] Entering Prompt run with input:
{
  "query": "谁是王尔德？仅从“小说”中找答案。"
}
[chain/end] [1:retriever:Retriever > 2:chain:RunnableSequence > 3:prompt:FewShotPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "base",
    "StringPromptValue"
  ],
  "kwargs": {
    "text": "Your goal is to structure the user's query to match the request schema provided below.\n\n\n\n<< Structured Request Schema >>\n\nWhen responding use a markdown code snippet with a JSON object formatted in the following schema:\n\n\n\n```json\n\n{\n\n    \"query\": string \\ text string to compare to document contents\n\n    \"filter\": string \\ logical condition statement for filtering documents\n\n}\n\n```\n\n\n\nThe qu


llama_print_timings:        load time =    6683.99 ms
llama_print_timings:      sample time =      93.84 ms /   256 runs   (    0.37 ms per token,  2728.11 tokens per second)
llama_print_timings: prompt eval time =     942.52 ms /    19 tokens (   49.61 ms per token,    20.16 tokens per second)
llama_print_timings:        eval time =   20549.64 ms /   255 runs   (   80.59 ms per token,    12.41 tokens per second)
llama_print_timings:       total time =   23202.09 ms /   274 tokens
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 

[llm/end] [1:retriever:Retriever > 2:chain:RunnableSequence > 4:llm:LlamaCpp] [23.21s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{  \n    \"query\": \"谁是王尔德？\",  \n    \"filter\": \"or(like(\\\"source\\\", \\\"小说\\\"), like(\\\"tags\\\", \\\"小说\\\"), like(\\\"name\\\", \\\"小说\\\"))\"  \n}\n```[PAD151645]\n[PAD151644][PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644

[Document(page_content='【作者简介】\n奥斯卡·王尔德（Oscar Wilde，1854年10月16日—1900年11月30日），出生于爱尔兰都柏林，19世纪英国（准确来讲是爱尔兰，但是当时由英国统治）最伟大的作家与艺术家之一，以其剧作、诗歌、童话和小说闻名，唯美主义代表人物，19世纪80年代美学运动的主力和90年代颓废派运动的先驱。\n王尔德的文学活动领域十分宽广。他既是诗人（1881年就有他的诗集问世），又写小说、童话（包括《快乐王子》、《石榴之家》、《阿瑟·萨维尔勋爵的罪行》三个集子以及他唯一的长篇小说《道连·葛雷的画像》，以上四种除《快乐王子及其他童话》一种成书于1888年外，其余三种均于1891年出版）。他还写过不少评论和随笔（较重要的有他自己选编的《意图集》以及《社会主义制度下人的灵魂》，均刊行于1891年）。但为他赢得最辉煌成功的要数1892——1895年间先后在伦敦西区舞台上首演的社会讽刺喜剧《温德米尔夫人的扇子》、《一个无足轻重的女人》、《一个理想的丈夫》和《你真的重要》。\n王尔德总共写过九部戏剧，但另外四部剧作已被遗忘。1895年2月14日，王尔德最后一部、也是他才华机智达到巅峰状态的剧作《认真的重要》在圣詹姆斯剧院首演，观众如潮，盛况空前。两周后，王尔德在阿尔比马尔俱乐部收到昆斯伯里侯爵约翰·道格拉斯留下的名片，上面写着：“致装模作样的好男色者奥斯卡·王尔德”。因为王尔德从1891年开始便与比他小十六岁的阿尔弗雷德·道格拉斯（侯爵之子，当时还在上牛津大学）有不正当关系，而且经常“俪影成双”地出现在伦敦的公共场所并一起旅游，侯爵十分反感。王尔德与年轻男性的同性恋行为遭人物议不自此时始。同年，王尔德以诽谤罪把昆斯伯里侯爵告上法庭，居然向他一贯通过自己的作品和生活方式加以百般嘲弄的维多利亚时代的英国法律和社会求助（两年后，王尔德沉痛地承认自己干了“一生中最可耻、最无法原谅、最可鄙的事 ”）。1895 年4月3日法院开庭审理王尔德诉昆斯伯里侯爵诽谤案（王尔德自幼渴望的倒是能在“女王诉王尔德”的官司中出庭），被告轻而易举地反证原告确系“好男色者”，从而推翻关于诽谤的指控。4月5日，侯爵无罪获释，王尔德反因涉嫌“同其他男子发生有伤风化的肉体关系”被捕后取保候审 。5月25日，伦敦中央刑事法院根据1885年通过的针对男子同性恋的刑法

In [21]:
%%time 

import langchain
langchain.debug = True

retriever.invoke('人生有几个不捡？仅从“笑死”中找答案。')

Llama.generate: prefix-match hit


[chain/start] [1:retriever:Retriever > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "query": "人生有几个不捡？仅从“笑死”中找答案。"
}
[chain/start] [1:retriever:Retriever > 2:chain:RunnableSequence > 3:prompt:FewShotPromptTemplate] Entering Prompt run with input:
{
  "query": "人生有几个不捡？仅从“笑死”中找答案。"
}
[chain/end] [1:retriever:Retriever > 2:chain:RunnableSequence > 3:prompt:FewShotPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "base",
    "StringPromptValue"
  ],
  "kwargs": {
    "text": "Your goal is to structure the user's query to match the request schema provided below.\n\n\n\n<< Structured Request Schema >>\n\nWhen responding use a markdown code snippet with a JSON object formatted in the following schema:\n\n\n\n```json\n\n{\n\n    \"query\": string \\ text string to compare to document contents\n\n    \"filter\": string \\ logical condition statement for filtering documents\n\n}\n\n```\n\n\n\nTh


llama_print_timings:        load time =    6683.99 ms
llama_print_timings:      sample time =      91.92 ms /   256 runs   (    0.36 ms per token,  2785.00 tokens per second)
llama_print_timings: prompt eval time =    1023.08 ms /    19 tokens (   53.85 ms per token,    18.57 tokens per second)
llama_print_timings:        eval time =   18236.48 ms /   255 runs   (   71.52 ms per token,    13.98 tokens per second)
llama_print_timings:       total time =   20953.50 ms /   274 tokens
Metadata key author not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key tags not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_start not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'dat

[llm/end] [1:retriever:Retriever > 2:chain:RunnableSequence > 4:llm:LlamaCpp] [20.96s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{  \n     \"query\": \"人生有几个不捡？\",  \n     \"filter\": \"or(like(\\\"source\\\", \\\"笑死\\\"), like(\\\"tags\\\", \\\"笑死\\\"), like(\\\"name\\\", \\\"笑死\\\"))\"  \n}\n```[PAD151645]\n[PAD151644]'t be able to solve this[PAD151645]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD1516

[Document(page_content='人生四不捡：塔吊下边冰红茶，铁轨上边牛肉干，过山车下八宝粥，课桌后边葡萄干', metadata={'id': 'doc:notiondb:ed2544cc5dc24ab0b7f921d300e2b010', 'author': None, 'name': '笑死', 'source': '格言小集', 'tags': None, 'date_start': None, 'date_end': None})]

In [22]:
%%time 

import langchain
langchain.debug = True

retriever.invoke('请列举NBA著名球星3名')

Llama.generate: prefix-match hit


[chain/start] [1:retriever:Retriever > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "query": "请列举NBA著名球星3名"
}
[chain/start] [1:retriever:Retriever > 2:chain:RunnableSequence > 3:prompt:FewShotPromptTemplate] Entering Prompt run with input:
{
  "query": "请列举NBA著名球星3名"
}
[chain/end] [1:retriever:Retriever > 2:chain:RunnableSequence > 3:prompt:FewShotPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "base",
    "StringPromptValue"
  ],
  "kwargs": {
    "text": "Your goal is to structure the user's query to match the request schema provided below.\n\n\n\n<< Structured Request Schema >>\n\nWhen responding use a markdown code snippet with a JSON object formatted in the following schema:\n\n\n\n```json\n\n{\n\n    \"query\": string \\ text string to compare to document contents\n\n    \"filter\": string \\ logical condition statement for filtering documents\n\n}\n\n```\n\n\n\nThe query string


llama_print_timings:        load time =    6683.99 ms
llama_print_timings:      sample time =      91.52 ms /   256 runs   (    0.36 ms per token,  2797.36 tokens per second)
llama_print_timings: prompt eval time =     653.46 ms /    13 tokens (   50.27 ms per token,    19.89 tokens per second)
llama_print_timings:        eval time =   18184.35 ms /   255 runs   (   71.31 ms per token,    14.02 tokens per second)
llama_print_timings:       total time =   20542.78 ms /   268 tokens
Metadata key author not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key author not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'dat

[llm/end] [1:retriever:Retriever > 2:chain:RunnableSequence > 4:llm:LlamaCpp] [20.55s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{  \n    \"query\": \"\",  \n    \"filter\": \"NO_FILTER\"\n}\n```[PAD151645]\n[PAD151644][PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD1

[Document(page_content='', metadata={'id': 'doc:notiondb:26995b7bf5434eabbd41856e6cb8cc5e', 'author': None, 'name': '中华诗歌百年精华', 'source': '读书笔记（文学）', 'tags': '诗', 'date_start': '20240228', 'date_end': None}),
 Document(page_content='', metadata={'id': 'doc:notiondb:c057a3ca679d441aac0475ca8a84d196', 'author': None, 'name': None, 'source': '格言小集', 'tags': None, 'date_start': None, 'date_end': None}),
 Document(page_content='【作者简介】\n奥斯卡·王尔德（Oscar Wilde，1854年10月16日—1900年11月30日），出生于爱尔兰都柏林，19世纪英国（准确来讲是爱尔兰，但是当时由英国统治）最伟大的作家与艺术家之一，以其剧作、诗歌、童话和小说闻名，唯美主义代表人物，19世纪80年代美学运动的主力和90年代颓废派运动的先驱。\n王尔德的文学活动领域十分宽广。他既是诗人（1881年就有他的诗集问世），又写小说、童话（包括《快乐王子》、《石榴之家》、《阿瑟·萨维尔勋爵的罪行》三个集子以及他唯一的长篇小说《道连·葛雷的画像》，以上四种除《快乐王子及其他童话》一种成书于1888年外，其余三种均于1891年出版）。他还写过不少评论和随笔（较重要的有他自己选编的《意图集》以及《社会主义制度下人的灵魂》，均刊行于1891年）。但为他赢得最辉煌成功的要数1892——1895年间先后在伦敦西区舞台上首演的社会讽刺喜剧《温德米尔夫人的扇子》、《一个无足轻重的女人》、《一个理想的丈夫》和《你真的重要》。\n王尔德总共写过九部戏剧，但另外四部剧作已被遗忘。1895年2月14日，王尔德最后一部、也是他才华机智达到巅峰状态的剧作《认真的重要》在圣詹姆斯剧院首演，观众如潮，盛况空前。两周后，王尔德在阿尔比马尔俱乐部收到昆斯伯里侯爵约翰·道格拉斯留下的名片，上面写着：“致装

In [23]:
import langchain
langchain.debug = False

q = '人生有几个不捡？仅从“笑死”中找答案。'

new_query, search_kwargs = retriever._prepare_query(
    q,
    structured_query=query_constructor.invoke({'query': q}),
)

print(new_query, str(search_kwargs['filter']))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    6683.99 ms
llama_print_timings:      sample time =      91.82 ms /   256 runs   (    0.36 ms per token,  2788.12 tokens per second)
llama_print_timings: prompt eval time =    1236.85 ms /    19 tokens (   65.10 ms per token,    15.36 tokens per second)
llama_print_timings:        eval time =   20111.53 ms /   255 runs   (   78.87 ms per token,    12.68 tokens per second)
llama_print_timings:       total time =   23073.00 ms /   274 tokens


人生有几个不捡？ (@source:(笑死) | (@tags:(笑死) | @name:(笑死)))


In [ ]:
# from notion_agent import chatbot

# llm = chatbot(
#     'Qwen/Qwen1.5-7B-Chat', 
#     _CONFIGS['model_path']+'/'+'qwen1_5-7b-chat-q4_0.gguf', 
#     **_CONFIGS['llm']
# )